# Partition function approach

## Goals
In this notebook, we will 

1. Justify the matricies to use for a/c transformations
2. Construct a set of transformations that cover the entire space of the cell shape (at least for bcc/omega/hcp Ti)
3. Create a workflow to run the structures
4. Analyze the results (plotting a 0K surface and finding the minima)
5. Construct a workflow to calculation vibrational properties of the selected minima
6. Explore how to extend this method (specifically deformations) to any pure element for all the structures of that element in the Materials Project

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pymatgen import Lattice, MPRester

/Users/brandon/.virtualenvs/atomate/lib/python2.7/site-packages/pymatgen/__init__.py:34: UserWarning: With effect from pmg 5.0, all pymatgen settings are prefixed with a "PMG_". E.g., "PMG_VASP_PSP_DIR" instead of "VASP_PSP_DIR".
  warnings.warn('With effect from pmg 5.0, all pymatgen settings are'


## 1. Justify the matricies for a/c transformations

We need to find the set of matricies that lets us adjust our a/c ratios, but maintain symmetry. The approach is to create hexagonal lattices with pymatgen, get the matrices and do an inversion and dot product to get the transformation matrix. For the `a` transformation:

In [2]:
hcp = Lattice.hexagonal(2,4)
a_deformed_hcp = Lattice.hexagonal(2.2 ,4) # 10% increased a hcp lattice
transformation_matrix = np.linalg.inv(hcp.matrix).dot(a_deformed_hcp.matrix)
print(transformation_matrix)

[[  1.10000000e+00   0.00000000e+00   6.12323400e-18]
 [  1.63781057e-17   1.10000000e+00   1.06057524e-17]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


Taking the very small numbers as 0, we can see that the transformation matrix for increasing `a` by 10% is:
```
[[  1.1  0.0  0.0]
 [  0.0  1.1  0.0]
 [  0.0  0.0  1.0]]
```
or more generally,
```
[[  1+x  0.0  0.0]
 [  0.0  1+x  0.0]
 [  0.0  0.0  1.0]]
```
where `x` is the deformation fraction.

Let's do the same for `c`:

In [3]:
c_deformed_hcp = Lattice.hexagonal(2 ,4.4) # 10% increased c hcp lattice
transformation_matrix = np.linalg.inv(hcp.matrix).dot(c_deformed_hcp.matrix)
print(transformation_matrix)

[[  1.00000000e+00   0.00000000e+00  -6.12323400e-18]
 [  3.43504199e-17   1.00000000e+00  -1.06057524e-17]
 [  0.00000000e+00   0.00000000e+00   1.10000000e+00]]


We can generalize this (with the rounding) to a a similar
```
[[  1.0  0.0  0.0]
 [  0.0  1.0  0.0]
 [  0.0  0.0  1+x]]
```

## 2. Construct a set of transformations that cover the entire space of the cell shape

Now that we know how to construct matricies that deform `a` and `c` alone, we can create a list of matrices that form a plane mesh of deformations over the entire `a`-`c` space. 

In [4]:
mpr = MPRester()
# mp-72 is P6/mmm
# mp-46 is P6_3/mmc
# mp-6985 is Fm-3m
# mp-73 is Im-3m
hcp_ti = mpr.get_structure_by_material_id('mp-46')
omega_ti = mpr.get_structure_by_material_id('mp-72')
bcc_ti = mpr.get_structure_by_material_id('mp-73')

First we should make sure we can get `a`-`c` limits for hcp/bcc. We'll make a supercell of bcc ([1,1,2]) to make the cell size comparable to the hcp (2 atoms in each). 

In [5]:
print('HCP a: {:0.3f}\nHCP c: {:0.3f}'.format(hcp_ti.lattice.a, hcp_ti.lattice.c))

bcc_ti_ss = bcc_ti.copy()
bcc_ti_ss.make_supercell([1,1,2])
print('BCC a: {:0.3f}\nBCC c: {:0.3f}'.format(bcc_ti_ss.lattice.a, bcc_ti_ss.lattice.c))

HCP a: 2.939
HCP c: 4.641
BCC a: 2.816
BCC c: 5.632


So we need to get `a` and `b` from 2.939 to 2.816 (0.958 fraction so we should decrease by at least -4.2% deformation would to sufficently deform `a`). For `c`, the hcp cell would need to increase from 4.641 to 5.631 (1.213 fraction, so at least a +21.3% deformation is required).

Now to compare hcp and omega, it takes a little more effort to get comparable cell sizes. Supercells of [3,3,2] and [2,2,3] to get 24 atoms in each cell.

In [6]:
hcp_ti_ss = hcp_ti.copy()
hcp_ti_ss.make_supercell([3,3,2])
print('HCP   a: {:0.3f}\nHCP   c: {:0.3f}'.format(hcp_ti_ss.lattice.a, hcp_ti_ss.lattice.c))

omega_ti_ss = omega_ti.copy()
omega_ti_ss.make_supercell([2,2,3])
print('Omega a: {:0.3f}\nOmega c: {:0.3f}'.format(omega_ti_ss.lattice.a, omega_ti_ss.lattice.c))

HCP   a: 8.817
HCP   c: 9.282
Omega a: 9.154
Omega c: 8.486


Similarly, we need to increase the `a` of HCP from 8.817 to 9.154 (+3.8%) and decrease the `c` from 9.282 to 8.486 (-8.6%). Thus the limits are conviently defined as:

### Deformation limits

The theoretical minimum limits, the proposed limits and proposed number of deformations are:

|   |  min |  max | |min | max | n  |
|:-:|:----:|:----:| |:---:|:--:|:---:|
| a | -4.2 |  3.8 | |-5  | 5  | 10  |
| c | -8.6 | 21.3 | |-10 | 25 | 20  |


## 3. Create a workflow to run the structures

We'll construct these deformations matricies, broadcast them to create a mesh of matricies and then create a workflow. Unlike before, we only need one workflow now to match all the cell sizes. 
# (**Is this true? What about ionic positions?**) 

In [17]:
# construct matricies
a_min = -0.05; a_max = 0.05; a_n = 10
a_defos = np.array([np.array([[1+x,0,0],[0,1+x,0],[0,0,1]]) for x in np.linspace(a_min, a_max, a_n)])
c_min = -0.10; c_max = 0.25; c_n = 20
c_defos = np.array([np.array([[1,0,0],[0,1,0],[0,0,1+x]]) for x in np.linspace(c_min, c_max, c_n)])

# broacast the deformation arrays and flatten them
broadcasted_defos = a_defos*c_defos[:,np.newaxis]
defos = broadcasted_defos.reshape(200, 3, 3)
print('{} deformed structures will be calculated.'.format(len(defos)))

# create the workflow and load it in launchpad
from atomate.vasp.workflows.base.deformations import get_wf_deformations
from atomate.vasp.powerups import add_modify_incar
from fireworks import LaunchPad

API_KEY = None # If None, will use the one from your ~/.pmgrc.yaml. Otherwise string of API_KEY
LAUNCHPAD_FILE = '/Users/brandon/.fireworks/my_launchpad.yaml' # If None, will try to autoload from FW_CONFIG_FILE environment variable. Otherwise string of file path
MP_ID = 'mp-46' # materials project id of the structure. mp-72 is the P6/mmm Ti
# mp-46 is P6_3/mmc
# mp-6985 is Fm-3m
# mp-73 is Im-3m

if API_KEY:
    mpr = MPRester(API_KEY)
else:
    mpr = MPRester()

if LAUNCHPAD_FILE:
    lpad = LaunchPad.from_file(LAUNCHPAD_FILE)
else:
    lpad = LaunchPad.auto_load()

# here the values in the linspace are the scaling factor for cell volume. E.g. -0.1 = 90%
# cubic_transformations = [np.eye(3)*((x+1)**(1.0/3.0)) for x in np.linspace(-0.1, 0.1, number_of_deformations)]

# construct the workflow
struct = hcp_ti
wf = get_wf_deformations(struct, defos, name="deformation")
wf = add_modify_incar(wf, modify_incar_params={"incar_update":{"SIGMA":0.2, "ISMEAR":1}})
lpad.add_wf(wf)

200 deformed structures will be calculated.
2017-03-15 11:54:49,541 INFO Added a workflow. id_map: {-1536: 1012, -1535: 1013, -1534: 1014, -1533: 1015, -1532: 1016, -1531: 1017, -1530: 1018, -1529: 1019, -1528: 1020, -1527: 1021, -1526: 1022, -1525: 1023, -1524: 1024, -1523: 1025, -1522: 1026, -1521: 1027, -1520: 1028, -1519: 1029, -1518: 1030, -1517: 1031, -1516: 1032, -1515: 1033, -1514: 1034, -1513: 1035, -1512: 1036, -1511: 1037, -1510: 1038, -1509: 1039, -1508: 1040, -1507: 1041, -1506: 1042, -1505: 1043, -1504: 1044, -1503: 1045, -1502: 1046, -1501: 1047, -1500: 1048, -1499: 1049, -1498: 1050, -1497: 1051, -1496: 1052, -1495: 1053, -1494: 1054, -1493: 1055, -1492: 1056, -1491: 1057, -1490: 1058, -1489: 1059, -1488: 1060, -1487: 1061, -1486: 1062, -1485: 1063, -1484: 1064, -1483: 1065, -1482: 1066, -1481: 1067, -1480: 1068, -1479: 1069, -1478: 1070, -1477: 1071, -1476: 1072, -1475: 1073, -1474: 1074, -1473: 1075, -1472: 1076, -1471: 1077, -1470: 1078, -1469: 1079, -1468: 1080, -14

{-1611: 937,
 -1610: 938,
 -1609: 939,
 -1608: 940,
 -1607: 941,
 -1606: 942,
 -1605: 943,
 -1604: 944,
 -1603: 945,
 -1602: 946,
 -1601: 947,
 -1600: 948,
 -1599: 949,
 -1598: 950,
 -1597: 951,
 -1596: 952,
 -1595: 953,
 -1594: 954,
 -1593: 955,
 -1592: 956,
 -1591: 957,
 -1590: 958,
 -1589: 959,
 -1588: 960,
 -1587: 961,
 -1586: 962,
 -1585: 963,
 -1584: 964,
 -1583: 965,
 -1582: 966,
 -1581: 967,
 -1580: 968,
 -1579: 969,
 -1578: 970,
 -1577: 971,
 -1576: 972,
 -1575: 973,
 -1574: 974,
 -1573: 975,
 -1572: 976,
 -1571: 977,
 -1570: 978,
 -1569: 979,
 -1568: 980,
 -1567: 981,
 -1566: 982,
 -1565: 983,
 -1564: 984,
 -1563: 985,
 -1562: 986,
 -1561: 987,
 -1560: 988,
 -1559: 989,
 -1558: 990,
 -1557: 991,
 -1556: 992,
 -1555: 993,
 -1554: 994,
 -1553: 995,
 -1552: 996,
 -1551: 997,
 -1550: 998,
 -1549: 999,
 -1548: 1000,
 -1547: 1001,
 -1546: 1002,
 -1545: 1003,
 -1544: 1004,
 -1543: 1005,
 -1542: 1006,
 -1541: 1007,
 -1540: 1008,
 -1539: 1009,
 -1538: 1010,
 -1537: 1011,
 -1536: 1012,

In [21]:
lpad.workflows.count()

82

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
2.93915881
2.93915881664
4.64117095
0.0
0.0
0.0
